In [ ]:
!pip install --quiet pyspark

In [ ]:
import numpy as np
import heapq
from heapq import heapify, heappush, heappop
import collections
import csv
import sys

In [ ]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("word count").setMaster("local")
sc = SparkContext(conf=conf)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_ds = '/content/drive/MyDrive/datasets/Datasets/train_set.csv'
test_ds =  '/content/drive/MyDrive/datasets/Datasets/test_set_small.csv'
# test_ds = '/content/drive/MyDrive/datasets/Datasets/test_set_medium.csv'
# test_ds = '/content/drive/MyDrive/datasets/Datasets/test_set_large1.csv'
# test_ds = '/content/drive/MyDrive/datasets/Datasets/test_set_large2.csv'

In [ ]:
mx_val_tr = []
mn_val_tr = []
mx_val_ts = []
mn_val_ts = []
numTrainSamples = 0
numTestSamples = 0
# opening the CSV file
with open(train_ds, mode ='r')as file:
   
  # reading the CSV file
  csvFile = csv.reader(file)
  enter = False

  # displaying the contents of the CSV file
  for row_list in csvFile:
      if(row_list[0][0].isdigit()):
        numTrainSamples += 1
        for i in range(0,len(row_list)):
          if(enter==False):
            mx_val_tr.append(float(row_list[i]))
            mn_val_tr.append(float(row_list[i]))
          else:
            mx_val_tr[i] = max(mx_val_tr[i],float(row_list[i]))
            mn_val_tr[i] = min(mn_val_tr[i],float(row_list[i]))

        enter = True

with open(test_ds, mode ='r')as file:
   
  # reading the CSV file
  csvFile = csv.reader(file)
  enter = False

  # displaying the contents of the CSV file
  for row_list in csvFile:
    if(row_list[0].isdigit()):
      numTestSamples += 1
      for i in range(1,len(row_list)):
        if(enter==False):
          mx_val_ts.append(float(row_list[i]))
          mn_val_ts.append(float(row_list[i]))
        else:
          mx_val_ts[i-1] = max(mx_val_ts[i-1],float(row_list[i]))
          mn_val_ts[i-1] = min(mn_val_ts[i-1],float(row_list[i]))    
      enter = True

print(mx_val_tr)
print(mn_val_tr)
print(mx_val_ts)
print(mn_val_ts)

[4.0, 13.0, 4.0, 13.0, 4.0, 13.0, 4.0, 13.0, 4.0, 13.0, 6.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
[4.0, 13.0, 4.0, 13.0, 4.0, 13.0, 4.0, 13.0, 4.0, 13.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [ ]:
def normalize_tr(row,mn_val_tr,mx_val_tr):
    row_list = row.split(",")
    list_ans = []
    if(row_list[0][0].isdigit()):
      list_ans.append(int(row_list[-1]))
      for i in range(0,len(row_list)):
        list_ans.append((float(row_list[i]) - mn_val_tr[i])/(mx_val_tr[i]-mn_val_tr[i]))
    return list_ans

def normalize_ts(row,mn_val_ts,mx_val_ts):
    # row_list = row[0].split(",")
    row_list = row.split(",")
    # print(row_list[0])
    list_ans = []
    if(row_list[0][0].isdigit()):
      list_ans.append(int(row_list[0]))
      for i in range(1,len(row_list)):
        list_ans.append((float(row_list[i]) - mn_val_ts[i-1])/(mx_val_ts[i-1]-mn_val_ts[i-1]))
    # return (list_ans,row[1])
    return list_ans

In [ ]:
# number_map = 10
# number_map = 50
# number_map = 100
number_map = 200
numFeatures = 10
tr_rdd_raw = sc.textFile(train_ds,number_map)
# ts_rdd_raw = sc.textFile(test_ds).zipWithIndex()
ts_rdd_raw = sc.textFile(test_ds)

# print(ts_rdd_raw.collect())
tr_rdd = tr_rdd_raw.map(lambda line : normalize_tr(line,mn_val_tr,mx_val_tr)).cache()
ts_rdd = ts_rdd_raw.map(lambda line : normalize_ts(line,mn_val_ts,mx_val_ts)).cache()

print(ts_rdd.take(100))

[[], [1, 0.0, 0.75, 0.3333333333333333, 0.08333333333333333, 0.6666666666666666, 0.16666666666666666, 0.6666666666666666, 0.5833333333333334, 0.0, 0.0], [2, 0.3333333333333333, 1.0, 0.6666666666666666, 0.3333333333333333, 0.6666666666666666, 0.5, 1.0, 0.4166666666666667, 0.0, 0.25], [3, 0.0, 0.16666666666666666, 0.0, 0.8333333333333334, 0.3333333333333333, 0.5833333333333334, 0.3333333333333333, 0.0, 0.3333333333333333, 0.25], [4, 0.0, 0.4166666666666667, 0.6666666666666666, 0.16666666666666666, 1.0, 0.5, 0.0, 0.5833333333333334, 0.6666666666666666, 0.8333333333333334], [5, 0.3333333333333333, 0.75, 0.6666666666666666, 0.25, 0.0, 0.4166666666666667, 0.3333333333333333, 0.9166666666666666, 0.3333333333333333, 0.4166666666666667], [6, 0.0, 0.25, 0.6666666666666666, 0.75, 0.3333333333333333, 0.8333333333333334, 0.3333333333333333, 0.4166666666666667, 0.0, 0.5], [7, 0.0, 0.75, 0.6666666666666666, 0.5833333333333334, 0.0, 0.25, 0.6666666666666666, 0.8333333333333334, 0.6666666666666666, 0.6

In [ ]:
def calIter(numTrainSmps,numTestSmps,memAllow):
  numIterations = 0;
  weightTrain = (8 * numTrainSmps * numFeatures) / (number_map * 1024.0 * 1024.0)
  weightTest = (8 * numTestSmps * numFeatures) / (1024.0 * 1024.0)
  print(weightTrain," ",weightTest)
  if (weightTrain + weightTest < memAllow * 1024.0):
        numIterations = 1
  else: 
    if (weightTrain >= memAllow * 1024.0):
      print("Train wight bigger than lim-task. Abort")
      sys.exit(1)

    numIterations = int((1 + (weightTest / ((memAllow * 1024.0) - weightTrain))))
  return numIterations

In [ ]:
avail_mem = 0.2
num_of_iter = calIter(numTrainSamples,numTestSamples,avail_mem)
# print(num_of_iter)
ts_rdd.partitionBy(num_of_iter)

0.0001903533935546875   5.7907867431640625


MapPartitionsRDD[452] at mapPartitions at PythonRDD.scala:145

In [ ]:
print(ts_rdd.take(100))

[[], [1, 0.0, 0.75, 0.3333333333333333, 0.08333333333333333, 0.6666666666666666, 0.16666666666666666, 0.6666666666666666, 0.5833333333333334, 0.0, 0.0], [2, 0.3333333333333333, 1.0, 0.6666666666666666, 0.3333333333333333, 0.6666666666666666, 0.5, 1.0, 0.4166666666666667, 0.0, 0.25], [3, 0.0, 0.16666666666666666, 0.0, 0.8333333333333334, 0.3333333333333333, 0.5833333333333334, 0.3333333333333333, 0.0, 0.3333333333333333, 0.25], [4, 0.0, 0.4166666666666667, 0.6666666666666666, 0.16666666666666666, 1.0, 0.5, 0.0, 0.5833333333333334, 0.6666666666666666, 0.8333333333333334], [5, 0.3333333333333333, 0.75, 0.6666666666666666, 0.25, 0.0, 0.4166666666666667, 0.3333333333333333, 0.9166666666666666, 0.3333333333333333, 0.4166666666666667], [6, 0.0, 0.25, 0.6666666666666666, 0.75, 0.3333333333333333, 0.8333333333333334, 0.3333333333333333, 0.4166666666666667, 0.0, 0.5], [7, 0.0, 0.75, 0.6666666666666666, 0.5833333333333334, 0.0, 0.25, 0.6666666666666666, 0.8333333333333334, 0.6666666666666666, 0.6

In [ ]:
def computekNN(row,all_ts,k):
  info = []
  if(len(row) > 0):
    point1 = np.array((row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10]))
    for i in range(0,len(all_ts)):
      if(len(all_ts[i])>0):
        point2 = np.array((all_ts[i][1],all_ts[i][2],all_ts[i][3],all_ts[i][4],all_ts[i][5],all_ts[i][6],all_ts[i][7],all_ts[i][8],all_ts[i][9],all_ts[i][10]))
        dist = np.linalg.norm(point1 - point2)

        info.append((all_ts[i][0],(dist,row[0])))
  return info

In [ ]:
def CombineResults(row):
  info = []
  max_heap = []
  heapq._heapify_max(max_heap)
  for i in row[1]:
    heappush(max_heap,(i[0],i[1]))
    if (len(max_heap) > k):
      heapq._heappop_max(max_heap)

  count = collections.defaultdict(lambda: 0)
  ans = -1
  cc = 0
  for val in max_heap:
    count[val[1]]+=1
    if(count[val[1]] > cc):
      cc = count[val[1]]
      ans = val[1]
  return (row[0],ans)

In [ ]:
broad_cast_data = ts_rdd.glom().collect()
# print(broad_cast_data)
k = 10
for ts_data in broad_cast_data:
  resultkNN = tr_rdd.flatMap(lambda line : computekNN(line,ts_data,k))
  res = resultkNN.groupByKey().mapValues(list)
  get_ans = res.map(lambda line : CombineResults(line))
  print(get_ans.take(1100))

[(202, 0), (404, 0), (606, 0), (808, 0), (1010, 0), (1212, 0), (1414, 0), (1616, 0), (1818, 0), (2020, 0), (2222, 1), (2424, 0), (2626, 1), (2828, 0), (3030, 0), (3232, 0), (3434, 0), (3636, 0), (3838, 0), (4040, 0), (4242, 0), (4444, 1), (4646, 0), (4848, 0), (5050, 0), (5252, 0), (5454, 0), (5656, 0), (5858, 0), (6060, 0), (6262, 0), (6464, 0), (6666, 0), (6868, 0), (7070, 0), (7272, 0), (7474, 0), (7676, 0), (7878, 0), (8080, 0), (8282, 1), (8484, 0), (8686, 0), (8888, 1), (9090, 0), (9292, 0), (9494, 0), (9696, 0), (9898, 0), (10100, 0), (10302, 0), (10504, 0), (10706, 0), (10908, 0), (11110, 0), (11312, 0), (11514, 0), (11716, 0), (11918, 0), (12120, 0), (12322, 0), (12524, 0), (12726, 1), (12928, 0), (13130, 0), (13332, 0), (13534, 1), (13736, 0), (13938, 0), (14140, 0), (14342, 0), (14544, 0), (14746, 0), (14948, 1), (15150, 1), (15352, 0), (15554, 0), (15756, 0), (15958, 0), (16160, 0), (16362, 0), (16564, 0), (16766, 1), (16968, 0), (17170, 0), (17372, 0), (17574, 0), (17776, 

In [ ]:
sc.stop()